<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# Capítulo 2 - Trabalhando com Texto

Pacotes que estão sendo usados neste notebook:

In [2]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.9 MB/s eta 0:00:00


In [3]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.5.1+cu124
tiktoken version: 0.9.0


- Este capítulo aborda a preparação de dados e a amostragem para deixar os dados de entrada "prontos" para o LLM.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">

## 2.1 Entendendo word embeddings

- Nenhum código nesta seção

- Existem muitas formas de embeddings; focamos em embeddings de texto neste livro.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

- Os LLMs trabalham com embeddings em espaços de alta dimensão (ou seja, milhares de dimensões)
- Como não conseguimos visualizar espaços de tão alta dimensionalidade (nós, humanos, pensamos em 1, 2 ou 3 dimensões), a figura abaixo ilustra um espaço de embedding bidimensional.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

## 2.2 Tokenizando texto

- Nesta seção, tokenizamos o texto, o que significa dividir o texto em unidades menores, como palavras individuais e caracteres de pontuação.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">

- Carregar o texto bruto com o qual queremos trabalhar
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) é um história curta de domínio público

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
mydrive = "/content/drive/My Drive/Colab Drive/llm"

In [7]:
import os
import urllib.request

if not os.path.exists(mydrive +"/the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = mydrive +"/the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

- (Se você encontrar um erro `ssl.SSLCertVerificationError` ao executar a célula de código anterior, pode ser devido ao uso de uma versão desatualizada do Python; você pode encontrar(https://github.com/rasbt/LLMs-from-scratch/pull/403))

In [9]:
with open(mydrive +"/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Número total de caracteres:", len(raw_text))
print(raw_text[:99])

Número total de caracteres: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- O objetivo é tokenizar e fazer o embedding deste texto para um LLM
- Vamos desenvolver um simples tokenizador com base em um texto de exemplo simples que podemos então aplicar ao texto acima.
- A seguinte expressão regular irá dividir nos espaços em branco.

In [10]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- Não queremos apenas dividir por espaços em branco, mas também por vírgulas e pontos, então vamos modificar a expressão regular para fazer isso também.

In [11]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- Como podemos ver, isso cria strings vazias, vamos removê-las.

In [12]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- Isso parece muito bom, mas vamos também lidar com outros tipos de pontuação, como pontos, sinais de interrogacão e assim por diante.

In [13]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- Isso é muito bom, e agora estamos prontos para aplicar essa tokenização ao texto bruto.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

In [14]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- Vamos calcular o número total de tokens

In [15]:
print(len(preprocessed))

4690


## 2.3 Convertendo tokens em token IDs

- Em seguida, convertemos os tokens de texto em token IDs que podemos processar através de camadas de embeddings mais tarde.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

- A partir desses tokens, agora podemos construir um vocabulário que consiste em todos os tokens únicos.

In [16]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [18]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- Abaixo estão as primeiras 50 entradas neste vocabulário:

In [19]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- Abaixo, ilustramos a tokenização de um breve texto de amostra usando um pequeno vocabulário:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">

- Colocando tudo isso agora em uma classe tokenizer

In [20]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- A função `encode` transforma texto em token IDs.
- A função `decode` transforma token IDs de volta em texto

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

- Podemos usar o tokenizador para codificar (ou seja, tokenizar) textos em inteiros.
- Esses inteiros podem se fazer embeddings (mais tarde) como entrada para o LLM.

In [22]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- Podemos decodificar os inteiros de volta em texto.

In [23]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [24]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 Adicionando tokens de contexto especiais

- É útil adicionar alguns tokens "especiais" para palavras desconhecidas e para denotar o fim de um texto.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

- Alguns tokenizadores usam tokens especiais para ajudar a LLM com contexto adicional.  
- Alguns desses tokens especiais são:  
  - `[BOS]` (beginning of sequence) marca o início do texto.  
  - `[EOS]` (end of sequence) indica onde o texto termina (geralmente usado para concatenar vários textos não relacionados, como dois artigos diferentes da Wikipedia ou dois livros distintos, por exemplo).  
  - `[PAD]` (padding) é utilizado quando treinamos LLMs com um tamanho de lote maior que 1 (incluímos vários textos com comprimentos diferentes; com o token de padding, preenchemos os textos mais curtos até o comprimento do maior texto, garantindo que todos tenham o mesmo tamanho).  
- `[UNK]` representa palavras que não estão incluídas no vocabulário.  

- Observe que o GPT-2 **não precisa de nenhum desses tokens mencionados acima**, usando apenas o token `<|endoftext|>` para reduzir a complexidade.  
- O `<|endoftext|>` é **análogo ao token `[EOS]`** mencionado acima.  
- O GPT também usa `<|endoftext|>` para padding (como normalmente utilizamos uma máscara ao treinar com entradas em lote, os tokens preenchidos não são atendidos de qualquer forma, então não importa quais tokens são usados).  
- O GPT-2 **não usa um token `<UNK>`** para palavras fora do vocabulário; em vez disso, ele utiliza um tokenizador **Byte-Pair Encoding (BPE)**, que divide palavras em **subunidades** — assunto que discutiremos em uma seção posterior.



- Usamos os tokens `<|endoftext|>` entre duas fontes independentes de texto.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

- Vamos ver o que acontece se tokenizarmos o seguinte texto:

In [25]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- O erro acima ocorre porque a palavra "Hello" não está contida no vocabulário.  
- Para lidar com esses casos, podemos adicionar tokens especiais como `"<|unk|>"` ao vocabulário para representar palavras desconhecidas.  
- Já que estamos expandindo o vocabulário, vamos adicionar outro token chamado `"<|endoftext|>"`, que é usado no treinamento do GPT-2 para indicar o fim de um texto (e também é utilizado entre textos concatenados, como quando nossos conjuntos de treinamento consistem em múltiplos artigos, livros, etc.).

In [26]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [27]:
len(vocab.items())

1132

In [28]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- Também precisamos ajustar o tokenizador adequadamente para que ele saiba quando e como usar o novo token `<unk>`.

In [29]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

- Vamos tentar tokenizar um texto com o tokenizador modificado.

In [30]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [31]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [32]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 Codificação Byte-Pair (BytePair Encoding - BPE)

- O GPT-2 usou a **codificação Byte-Pair (BPE)** como seu tokenizador.  
- Isso permite que o modelo divida palavras que não estão em seu vocabulário predefinido em **subunidades menores** ou até mesmo **caracteres individuais**, possibilitando o tratamento de palavras fora do vocabulário (*out-of-vocabulary words*).  
- Por exemplo, se o vocabulário do GPT-2 **não contém** a palavra `"unfamiliarword"`, ele pode tokenizá-la como `["unfam", "iliar", "word"]` ou alguma outra divisão de subpalavras, dependendo das regras aprendidas pelo BPE.  
- O tokenizador BPE original pode ser encontrado aqui: [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py).  
- Neste capítulo, estamos utilizando o **tokenizador BPE** da biblioteca de código aberto da OpenAI, chamada **[tiktoken](https://github.com/openai/tiktoken)**, que implementa seus algoritmos principais em **Rust**, melhorando o desempenho computacional.  
- Eu criei um **notebook** na pasta [./bytepair_encoder](../02_bonus_bytepair-encoder) que compara essas duas implementações lado a lado (**o `tiktoken` foi cerca de 5x mais rápido no texto de exemplo**).  

In [33]:
!pip install tiktoken

In [34]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [35]:
tokenizer = tiktoken.get_encoding("gpt2")

In [36]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [37]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- Tokenizadores BPE dividem palavras desconhecidas em subpalavras e caracteres individuais.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">

## 2.6 Amostragem de dados com uma janela deslizante

- Treinamos LLMs para gerar uma palavra por vez, portanto, queremos preparar os dados de treinamento de forma adequada, onde a próxima palavra em uma sequência representa o alvo a ser previsto.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

In [38]:
with open(mydrive +"/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


- Para cada fragmento de texto, queremos definir as entradas e os alvos.  
- Como queremos que o modelo preveja a próxima palavra, os alvos são as entradas deslocadas uma posição para a direita.

In [39]:
enc_sample = enc_text[50:]

In [40]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- Um por um, a previsão ficaria assim:

In [41]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [42]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- Cuidaremos da previsão da próxima palavra em um capítulo posterior, após abordarmos o attention mechanism.  
- Por enquanto, implementamos um dataloader simples que percorre o dataset de entrada e retorna as entradas e os alvos deslocados em uma posição.

- Instale e importe o PyTorch (consulte o **Apêndice A** para dicas de instalação).

In [43]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.5.1+cu124


- Usamos uma abordagem de janela deslizante, alterando a posição em +1.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

- Crie um dataset e um dataloader que extraia fragmentos do dataset de texto de entrada.

In [47]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize todo o texto
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use uma janela deslizante para dividir o livro em sequências sobrepostas de comprimento máximo (`max_length`).
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [44]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

- Vamos testar o **dataloader** com um tamanho de lote (**batch size**) de 1 para uma **LLM** com um tamanho de contexto de 4.

In [45]:
with open(mydrive +"/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [48]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [49]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- Um exemplo usando `stride` igual ao tamanho do contexto (aqui: 4), conforme mostrado abaixo:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

- Também podemos criar saídas em **lotes** (*batched outputs*).  
- Observe que aumentamos o **`stride`** aqui para evitar sobreposição entre os lotes, pois um maior nível de sobreposição pode levar a um **overfitting** aumentado.

In [50]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 Criando embeddings de tokens

- Os dados já estão quase prontos para uma **LLM**.  
- Mas, por fim, vamos **converter os tokens em representações vetoriais contínuas** usando uma camada de embedding.  
- Normalmente, essas **camadas de embedding** fazem parte da própria LLM e são **atualizadas (treinadas) durante o treinamento do modelo**.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

- Suponha que temos os seguintes quatro exemplos de entrada com **IDs de entrada** 2, 3, 5 e 1 (após a tokenização):

In [ ]:
input_ids = torch.tensor([2, 3, 5, 1])

- Para simplificar, suponha que temos um **vocabulário pequeno de apenas 6 palavras** e queremos criar **embeddings de tamanho 3**.

In [51]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- Isso resultaria em uma matriz de pesos 6×3.

In [54]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- Para aqueles que estão familiarizados com **one-hot encoding**, a abordagem da camada de embedding acima é essencialmente uma forma mais eficiente de implementar one-hot encoding seguido por **multiplicação de matrizes** em uma camada totalmente conectada, conforme descrito no código suplementar em [./embedding_vs_matmul](../03_bonus_embedding-vs-matmul).  
- Como a camada de embedding é apenas uma implementação mais eficiente, equivalente à abordagem de one-hot encoding seguida de multiplicação de matrizes, ela pode ser vista como uma camada de rede neural que pode ser otimizada via backpropagation.

- Para converter um token com id 3 em um vetor tridimensional, fazemos o seguinte:

In [ ]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- Observe que o exemplo acima corresponde à quarta linha na matriz de pesos da `embedding_layer`.  
- Para incorporar todos os quatro valores de `input_ids` mencionados acima, fazemos o seguinte:

In [ ]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- Uma camada de embedding é essencialmente uma operação de busca.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

- Você pode se interessar pelo conteúdo bônus comparando camadas de embedding com camadas lineares regulares: [../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul).

## 2.8 Codificando posições das palavras

- Embedding layer convert IDs into identical vector representations regardless of where they are located in the input sequence:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

- Positional embeddings are combined with the token embedding vector to form the input embeddings for a large language model:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

- The BytePair encoder has a vocabulary size of 50,257:
- Suppose we want to encode the input tokens into a 256-dimensional vector representation:

In [ ]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- If we sample data from the dataloader, we embed the tokens in each batch into a 256-dimensional vector
- If we have a batch size of 8 with 4 tokens each, this results in a 8 x 4 x 256 tensor:

In [ ]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [ ]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [ ]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(token_embeddings)

torch.Size([8, 4, 256])


- GPT-2 uses absolute position embeddings, so we just create another embedding layer:

In [ ]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# uncomment & execute the following line to see how the embedding layer weights look like
# print(pos_embedding_layer.weight)

In [ ]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(pos_embeddings)

torch.Size([4, 256])


- To create the input embeddings used in an LLM, we simply add the token and the positional embeddings:

In [ ]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(input_embeddings)

torch.Size([8, 4, 256])


- In the initial phase of the input processing workflow, the input text is segmented into separate tokens
- Following this segmentation, these tokens are transformed into token IDs based on a predefined vocabulary:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

# Summary and takeaways

See the [./dataloader.ipynb](./dataloader.ipynb) code notebook, which is a concise version of the data loader that we implemented in this chapter and will need for training the GPT model in upcoming chapters.

See [./exercise-solutions.ipynb](./exercise-solutions.ipynb) for the exercise solutions.

See the [Byte Pair Encoding (BPE) Tokenizer From Scratch](../02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb) notebook if you are interested in learning how the GPT-2 tokenizer can be implemented and trained from scratch.